In [1]:
import torchdata.datapipes as dp
import torchtext.transforms as T 

import spacy
import torchtext

import pandas as pd

In [2]:
def create_dataiter(filepath, skip_lines=1, delimiter=','):
    data_pipe = dp.iter.IterableWrapper([filepath])
    data_pipe = dp.iter.FileOpener(data_pipe, mode='rb', length=0)
    data_pipe = data_pipe.parse_csv(skip_lines=skip_lines, delimiter=delimiter, as_tuple=True)
    return data_pipe

In [3]:
train_og = create_dataiter('../data/processed/train_og.csv')
train_aug = create_dataiter('../data/processed/train_aug.csv')
train_fin = create_dataiter('../data/processed/train_final.csv')
test = create_dataiter('../data/processed/test.csv')

In [4]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchdata.datapipes.iter import ShardingFilter

tokenizer = get_tokenizer("spacy", language='pt')

def yield_tokens(data_iter):
        for text, _, j in data_iter:
            yield tokenizer(text)

train_iter = ShardingFilter(iter([*train_og]))
test_iter = ShardingFilter(iter([*test]))

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["[UNK]"])
vocab.set_default_index(vocab["[UNK]"])

text_pipeline = lambda word: vocab(tokenizer(word))
label_pipeline = lambda idx: int(idx)
labelinf_pipeline = lambda idx: int(idx) + 1

c:\Users\kevin\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchtext\data\utils.py:105: UserWarning: Spacy model "pt" could not be loaded, trying "pt_core_news_sm" instead
  warnings.warn(


In [5]:
text_pipeline('biologia historia')

[5, 1]

In [6]:
from torch.utils.data import DataLoader
import torch
from torchdata.datapipes.iter import IterableWrapper, ShardingFilter

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_inf_list, label_list, text_list, offsets = [], [], [], [0]

    # Iterate through the batch
    for _text, _label_inf, _label in batch:
        # Process label_inf using labelinf_pipeline
        label_inf_list.append(labelinf_pipeline(_label_inf))
        
        # Process label using label_pipeline
        label_list.append(label_pipeline(_label))
        
        # Process text using text_pipeline and convert to a tensor
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        
        # Update offsets to keep track of sequence lengths
        offsets.append(processed_text.size(0))

    # Convert the lists to PyTorch tensors and move to the device
    label_inf_list = torch.tensor(label_inf_list, dtype=torch.int64).to(device)
    label_list = torch.tensor(label_list, dtype=torch.int64).to(device)
    
    # Calculate cumulative offsets
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0).to(device)
    
    # Concatenate the text tensors
    text_list = torch.cat(text_list).to(device)
    
    # Return the processed data
    return label_inf_list, label_list, text_list, offsets


train_iter = ShardingFilter(iter([*train_og]))
dataloader = DataLoader(train_iter, shuffle=False, batch_size=16, collate_fn=collate_batch)


In [7]:
from SentenceClassification import sentence_models as sc

In [8]:
train_iter = ShardingFilter(iter([*train_og]))

num_class = len(set([label for (label) in train_iter]))
vocab_size = len(vocab)
emsize = 16
model = sc.LSTM(vocab_size = len(vocab),
                embedding_dim = 30, 
                hidden_size = 15, 
                num_layers = 30, 
                num_classes = 7).to(device)

In [9]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    
    for idx, (label_inf, label, text, offsets) in enumerate(dataloader):
        if idx == 26:
            continue
        optimizer.zero_grad()
        predicted_label = model(text)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 1

    with torch.no_grad():
        for idx, (label_inf, label, text, offsets) in enumerate(dataloader):
            if text.size(0) != label.size(0):
                continue  # Skip the current batch
            predicted_label = model(text)

            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [10]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# Hyperparameters
EPOCHS = 10  # epoch
LR = 1  # learning rate
BATCH_SIZE = 30  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)

train_iter = ShardingFilter(iter([*train_og]))
test_iter = ShardingFilter(iter([*test]))

total_accu = None
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.90)
split_train_, split_valid_ = random_split(
    train_dataset, [num_train, len(train_dataset) - num_train]
)

train_dataloader = DataLoader(
    split_train_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
valid_dataloader = DataLoader(
    split_valid_, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)
test_dataloader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_batch
)


In [11]:
for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  0.87s | valid accuracy    0.000 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  0.95s | valid accuracy    0.000 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  0.82s | valid accuracy    0.000 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  0.81s | valid accuracy    0.000 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  0.79s | valid accuracy    0.000 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  0.82s |

In [24]:
import torch.nn as nn


class LSTM(nn.Module):
    """
    A class representing an LSTM model.

    Attributes:
    - hidden_size (int): The size of the hidden tensor.
    - num_layers (int): The number of LSTM layers.
    - embedding (nn.Embedding): An embedding layer to convert words into dense vectors.
    - lstm (nn.LSTM): An LSTM layer.
    - fc (nn.Linear): A linear layer for classification.
    """
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers, num_classes):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # Embedding layer to convert words into dense vectors
        self.embedding = nn.Embedding(vocab_size, embedding_dim, sparse=False)
        
        # LSTM layer
        self.lstm = nn.LSTM(embedding_dim, hidden_size, num_layers, batch_first=True)
        
        # Fully connected layer for classification
        self.fc = nn.Linear(hidden_size, num_classes)

        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        
    def forward(self, x):
        """
        Performs a forward pass through the LSTM module.

        Args:
        - x (torch.Tensor): The input tensor.

        Returns:
        - out (torch.Tensor): The output tensor.
        """
        # Input x should be of shape (batch_size, sequence_length)
        embedded = self.embedding(x)  # Embedding the input

        output, (hidden, cell) = self.lstm(embedded)
        # output dim: [sentence length, batch size, hidden dim]
        # hidden dim: [1, batch size, hidden dim]

        hidden.squeeze_(0)
        # hidden dim: [batch size, hidden dim]
        
        output = self.fc(hidden)
        return output

        # # Initialize hidden state with zeros
        # h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        # c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        
        # # Forward pass through LSTM
        # out, _ = self.lstm(embedded, (h0, c0))
        
        # # Get the output of the last time step
        # out = out[:, -1, :]
        
        # # Fully connected layer
        # out = self.fc(out)
        # return out

In [25]:
model = LSTM(vocab_size = len(vocab),
             embedding_dim = 20, 
             hidden_size = 15, 
             num_layers = 30, 
             num_classes = 7).to(device)